In [ ]:
!pip install -q google-generativeai datasets

In [ ]:
import google.generativeai as genai

In [ ]:
import os
if 'COLAB_RELEASE_TAG' in os.environ:
  from google.colab import userdata
  import pathlib
  pathlib.Path('client_secret.json').write_text(userdata.get('CLIENT_SECRET'))

  # Use `--no-browser` in colab
  !gcloud auth application-default login --no-browser --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'
else:
  !gcloud auth application-default login --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'

You are authorizing client libraries without access to a web browser. Please run the following command on a machine with a web browser and copy its output back here. Make sure the installed gcloud version is 372.0.0 or newer.

gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=638561879635-kdmc4pvldeq51cbn10rknlrrgulf1vvv.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.tuning&state=E9UvoBq5wjNhwgjigt0jJVYi4lpgjs&access_type=offline&code_challenge=ZfdhlCtH3c7eD4Y-BaHNItaH9Kg2GrDdeYCUmGPSgnA&code_challenge_method=S256&token_usage=remote"


Enter the output of the above command: https://localhost:8085/?state=E9UvoBq5wjNhwgjigt0jJVYi4lpgjs&code=4/0AcvDMrBDQ68OONrI4o2-TKXtp2hZ4IjOG1J1Mdzf0PghN2_IpcocBOGZQcRhlFHSpFLXHg&scope=https://www.googleapis.com/auth/cloud-platform%20https://www.googleapis.com/auth/generati

In [ ]:
from datasets import load_dataset

dataset = load_dataset("krishanu23/text-to-python")
print(dataset)

In [ ]:
dataset['train'][0]

{'querry': 'generate a bar plot for sold prices with different location.',
 'code output': "import seaborn as sns\\nsns.barplot(x='Location', y='Sold Price', data=df)"}

In [ ]:
df = dataset['train'].to_pandas()
df.columns = ['input','output']

In [ ]:
def formatter(x):
 text = f"""\
You are working with a pandas dataframe in Python. The name of the dataframe is `df`. Given an input querry synthesis a python code for that.


Input:
{x['input']}


Output:
 """
 return text


df['text_input'] = df.apply(formatter,axis=1)
print(df['text_input'][0])

You are working with a pandas dataframe in Python. The name of the dataframe is `df`. Given an input querry synthesis a python code for that.


Input:
generate a bar plot for sold prices with different location.


Output:
 


In [ ]:
bm_name = "models/gemini-1.0-pro-001"
name = 'fine-tunned-10-epoch'
operation = genai.create_tuned_model(
   source_model=bm_name,
   training_data=df,
   id = name,
   epoch_count = 10,
   batch_size=4,
   learning_rate=0.001,
)

In [ ]:
model = genai.get_tuned_model(f'tunedModels/{name}')
print(model)

TunedModel(name='tunedModels/fine-tunned-10-epoch',
           source_model='models/gemini-1.0-pro-001',
           base_model='models/gemini-1.0-pro-001',
           display_name='',
           description='',
           temperature=0.9,
           top_p=1.0,
           top_k=0,
           state=<State.CREATING: 1>,
           create_time=datetime.datetime(2024, 8, 6, 13, 34, 31, 857027, tzinfo=datetime.timezone.utc),
           update_time=datetime.datetime(2024, 8, 6, 13, 34, 31, 857027, tzinfo=datetime.timezone.utc),
           tuning_task=TuningTask(start_time=datetime.datetime(2024, 8, 6, 13, 34, 34, 851696, tzinfo=datetime.timezone.utc),
                                  complete_time=None,
                                  snapshots=[],
                                  hyperparameters=Hyperparameters(epoch_count=10,
                                                                  batch_size=4,
                                                                  learning_rate=0.0

In [ ]:
print(operation.metadata)

total_steps: 158
tuned_model: "tunedModels/fine-tunned-10-epoch"



In [ ]:
import time


for status in operation.wait_bar():
 time.sleep(10)

  0%|          | 0/158 [00:00<?, ?it/s]